In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm
import pandas as pd

MODES = {
    "unigram" : (1, 1),
    "bigram" : (2, 2),
    "uni-bigram" : (1, 2)
}


class NaiveBayes:
    def __init__(self, mode: str = "unigram", stop_words = None) -> None:
        """
        Initialize NaiveBayes model

        Args:
            mode (str, optional): Mode for BagOfWords, should be either Unigram or Bigram. Defaults to "unigram".
            stop_words (list, optional): Stop words to eliminate from BagOfWords
        """
        self.count_vector = None
        self.probability_dict = dict()

        assert mode in MODES.keys(), "Mode should be either bigram or unigram"
        self.ngram_mode = MODES[mode]
        self.stop_words = stop_words

    def fit(self, x: np.ndarray, y: np.ndarray) -> None:
        """
        Fit the data, calculate probabilities according to it

        Args:
            x (np.ndarray): data with shape (N,1), each row consists of a mail
            y (np.ndarray): data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        """

        # columns is list of every words without their counts, counts stores in count_vector
        self.count_vector, columns = self.__vectorizer(x)

        self.__calculate_class_prior(y)     # calculates P(spam) and P (ham), and adds them into probability_dict
        # calculates P(x(i)|spam) and P(x(i)|ham) values, and adds them into probability_dict
        self.__calculate_likelihoods(y, columns, 1)

    def predict(self, x_predict: np.ndarray) -> np.ndarray:
        """
        Predict the labels of the mails in x_predict

        Args:
            x_predict (np.ndarray): Data to be predicted, shape (N,1)

        Returns:
            y_predict (np.ndarray): Predictions for the mails, shape (N,)
        """

        n = x_predict.shape[0]
        y_predict = np.zeros(n)

        vector, columns = self.__vectorizer(x_predict)

        for i in range(n):
            probability_of_spam = 0
            probability_of_ham = 0

            word_idx = np.arange(columns.shape[0])[vector[i] > 0] # only work on words that the text has
            # calculate P(vj | text)
            for j in word_idx:
                if "%s|spam" % columns[j] in self.probability_dict.keys():
                    probability_of_spam += vector[i][j] * self.probability_dict[columns[j] + "|spam"]
                    probability_of_ham += vector[i][j] * self.probability_dict[columns[j] + "|ham"]

            probability_of_spam += self.probability_dict["spam"]
            probability_of_ham += self.probability_dict["ham"]

            y_predict[i] = 1 if probability_of_spam > probability_of_ham else 0

        return y_predict

    def __vectorizer(self, arr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """
        Creates a matrix which stores the words that emails includes and their counts in each email

        :param arr: an numpy array with shape (N,1) for
        :return: a CountVectorizer matrix (N, number of different words in emails) and a vector named columns
        with shape (number of different words in emails,1) which stores all words appears in mails
        """

        # initializes CountVectorizer item with ngram_mode
        vectorizer = CountVectorizer(ngram_range=self.ngram_mode, stop_words=self.stop_words)

        # vector that holds all words in emails and their counts for each item
        vector = vectorizer.fit_transform(arr)

        # convert vector variable to array for usability
        count_vector = vector.toarray()

        # names of columns
        columns = vectorizer.get_feature_names_out()

        return count_vector, columns

    def __calculate_class_prior(self, y: np.ndarray) -> None:
        """
        Calculates class probabilities [P(spam) and P(ham)] for training examples, and adds the result into
        self.probability dictionary as "spam" and "ham" labels

        :param y: data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        :return: None
        """

        # labels are only 0 and 1 therefore if we sum all items we get number of 1s
        # instead of a for loop we can use this method
        number_of_spam = np.sum(y)
        number_of_ham = len(y) - number_of_spam

        self.probability_dict["spam"] = np.log(number_of_spam / y.shape[0])  # P(spam) = number of spams / N
        self.probability_dict["ham"] = np.log(number_of_ham / y.shape[0])     # P(ham) = number of hams / N

    def __calculate_likelihoods(self, y: np.ndarray, columns: np.ndarray, alpha: int) -> None:
        """
        Calculates likelihoods of each word that contains in emails as P(word|spam) and P(word|ham), and adds the results
        into self.probability dictionary as "word|spam" and "word|ham" labels

        :param y: data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        :param columns: a vector with shape (number of different words in emails,1) which stores all words appears in mails
        :param alpha: int value for smoothing
        :return: None
        """

        N, D = self.count_vector.shape
        spam_vector, ham_vector = np.sum(self.count_vector[y == 1], axis=0), np.sum(self.count_vector[y == 0], axis=0)

        n_s = np.sum(spam_vector) # | Text_spam |
        n_h = np.sum(ham_vector) # | Text_ham  |

        for word_i in range(D):
            n_w_s = spam_vector[word_i]
            n_h_s = ham_vector[word_i]

            self.probability_dict["%s|spam" % columns[word_i]] = np.log((n_w_s + alpha) / (n_s + D))
            self.probability_dict["%s|ham" % columns[word_i]] = np.log((n_h_s + alpha) / (n_h + D))

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

file_path = os.path.join(os.getcwd(), "emails.csv")
df = pd.read_csv(file_path)
X, y = df["text"].to_numpy(), df["spam"].to_numpy()

model = NaiveBayes()
model.fit(X, y)

## 3. Part 3

### 3.a. Analyzing effect of the words on prediction

To analyze the effects of the words on prediction we created a pipeline that works as follows:
1. We first eliminate the words in the word set according to their tfidf values for each document class (spam, ham). This step is composed of few steps.
    1. Compute tfidf values by using `TfidfTransformer`, we got a matrix of shape (N, D) (where N is equal to sample size and D equal to distinct word count). However we want an output that acts as a scoring metric among words so that we can eliminate among them.
    2. Take sum among `axis = 0`, sum by column, this way our matrix is reduced to shape (D,).
    3. Take the maximum 100 and minimum 100 tfidf valued words. These 200 words will the candidates for finding the words that suggests certain class by their presence/absence. To be accurate, we used minimum 100 tfidf values to be absence candidates and maximum 100 words to be presence candidates.
2. At this step, we see that our saved matrices **contains some stop words**.  To analyze the effects of presence and absence of a word, we compute their posterior probability as follows:
$$P(v_d|w) = \frac{P(w|v_d) * P(v_d)}{P(w)} \text{ where $P(v_d|w)$ is the probability that the class of the document is $v_d$ given that it contains $w$}$$
<center>and
$$P(v_d|\neg w) = \frac{P(w|v_d) * P(v_d)}{P(\neg w)} \text{ where $P(v_d|\neg w)$ is the probability that the class of the document is $v_d$ given that it doesn't contains $w$}$$

From our $\text{N$\dot{a}$ive Bayes}$ model, we know the conditional probilities $P(w|v_d)$, $P(\neg w|v_d) = 1 - P(w|v_d)$ and prior probilities $P(\text{spam})$, $P(\text{ham})$. Hence, we only need to calculate $P(w)$ and $P(\neg w)$. To calculate those we count the total occurance and divide it by total word count.

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

def find_presence_absence_probilities (X : np.ndarray, y : np.ndarray, stop_words = None):
    ## Create pipelines for two different document class
    pipe_spam = Pipeline([('count', CountVectorizer(stop_words = stop_words)),
                               ('tfidf', TfidfTransformer(use_idf = True, smooth_idf = True))]).fit(X[y==1])
    pipe_ham = Pipeline([('count', CountVectorizer(stop_words = stop_words)),
                               ('tfidf', TfidfTransformer(use_idf = True, smooth_idf = True))]).fit(X[y==0])
    ## Step 1. Eliminate words that has low tfidf values

    y_vals = [(1, pipe_spam), (0, pipe_ham)]
    y_words = dict()
    for y_val, pipe in y_vals:
        tfidf_vals = pipe.transform(X[y==y_val])
        words = pipe["count"].get_feature_names_out()
        sums = np.sum(tfidf_vals, axis=0).A1
        idx = np.argsort(sums)

        print("##### Highest TFIDF values for class %s #####" % y_val)
        print(words[idx[-100:]])
        y_words["y = %d" % y_val] = (idx[:100], idx[-100:])
    
    ## Step 2. Eliminate words that has low tfidf values
    vectorizer = CountVectorizer()
    counts = vectorizer.fit_transform(X)
    presence_counts = counts.sum(axis=0).A1
    total = counts.sum()

    doc_classes = [("spam", pipe_spam),
                   ("ham", pipe_ham)]
    word_p_a_dict = dict()
    N = X.shape[0]
    for doc_class, pipe in tqdm(doc_classes, desc="Calculating probilities for P(class | word in doc) and P(class | word not in doc)"):
        word_p_a_dict["%s absence" % doc_class] = []
        word_p_a_dict["%s presence" % doc_class] = []
        label = 1 if doc_class == "spam" else 0

        low_tfidf, high_tfidf = y_words["y = %d" % label]
        class_prior = model.probability_dict[doc_class]
        words = pipe["count"].get_feature_names_out()

        for word_idx in tqdm(high_tfidf, leave = False, desc="Doing presence %s" % doc_class):
            word = words[word_idx]
            word_idx_count = np.where(vectorizer.get_feature_names_out() == word)[0][0]
            cond_prob = model.probability_dict["%s|%s" % (word, doc_class)]
            presence_count = presence_counts[word_idx_count]
            p1 = (cond_prob + class_prior) - np.log(presence_count / total)
            word_p_a_dict["%s presence" % doc_class].append((words[word_idx], np.exp(p1)))

        for word_idx in tqdm(low_tfidf, leave = False, desc="Doing absence %s" % doc_class):
            word = words[word_idx]
            word_idx_count = np.where(vectorizer.get_feature_names_out() == word)[0][0]

            cond_prob = np.log(1 - np.exp(model.probability_dict["%s|%s" % (word, doc_class)]))
            presence_count = presence_counts[word_idx_count]
            p1 = cond_prob + class_prior - np.log((total-presence_count) / total)
            word_p_a_dict["%s absence" % doc_class].append((words[word_idx], np.exp(p1)))
    
    return word_p_a_dict
    

In [4]:
word_p_a_dict = find_presence_absence_probilities(X, y)

##### Highest TFIDF values for class 1 #####
['visit' 'interested' 'how' 'marketing' 'way' 'may' 'was' 'home' 'net'
 'about' 'start' '2005' 'receive' 'offer' '10' 'offers' '000' 'time' 'has'
 'search' 'only' 'account' 'want' 'but' 'list' 'life' 'what' 'into'
 'viagra' 'see' 'new' 'us' 'any' 'like' 'over' 'www' 'site' 'message'
 'out' 'make' 'logo' '95' 'information' 'mail' 'one' 'online' 'best'
 'please' 'need' 'more' 'save' 'free' 'my' 'no' 'do' 'money' 'now' 'an'
 'just' 'company' 'http' 'get' 'email' 'can' 'if' 'all' 'adobe' 'at' 'as'
 'click' 'by' 'website' 'business' 'software' 'com' 'here' 'on' 'will'
 'or' 'not' 'have' 'are' 'that' 'with' 'from' 'subject' 'be' 'our' 'it'
 'we' 'this' 'for' 'is' 'in' 'of' 'your' 'and' 'you' 'the' 'to']
##### Highest TFIDF values for class 0 #####
['regards' 'him' 'his' 'email' 'forward' '713' 'but' 'up' 'call' 'resume'
 'corp' 'some' 'information' 'need' 'get' 'crenshaw' '04' 'so' 'power'
 '12' 'very' 'conference' 'request' 'stinson' 'th' '11' 'm

Calculating probilities for P(class | word in doc) and P(class | word not in doc):   0%|          | 0/2 [00:00…

Doing presence spam:   0%|          | 0/100 [00:00<?, ?it/s]

Doing absence spam:   0%|          | 0/100 [00:00<?, ?it/s]

Doing presence ham:   0%|          | 0/100 [00:00<?, ?it/s]

Doing absence ham:   0%|          | 0/100 [00:00<?, ?it/s]

From the outputs of above cell, we note that highest tfidf values contains some of the stop words such as `is`, `on`, `be`, `of` etc. In below cells, we calculate the posterior probilities similar to how we did in our $\text{N$\dot{a}$ive Bayes}$ model. However, since we saved our prior probilities and conditional probilities in log form we continue to do our calculations in log form. Below we show the formulation:
$$ln{P(v_d|w)} = ln{\frac{P(w|v_d) * P(v_d)}{P(w)}} \tag{1}$$
$$ln{P(v_d|w)} = ln{P(w|v_d)} + ln{P(v_d)} - ln{P(w)} \tag{2}$$
<br>
<center>and
<br>
$$ln{P(v_d|\neg w)} = ln{\frac{P(\neg w|v_d) * P(v_d)}{P(\neg w)}} \tag{3}$$
$$ln{P(v_d|\neg w)} = ln{P(\neg w|v_d)} + ln{P(v_d)} - ln{P(\neg w)} \tag{4}$$

In [5]:
take_n = 10 # take n words with highest posterior probility
for key, value in word_p_a_dict.items():
    word_p_a_dict[key].sort(key= lambda x : x[1])
    word_p_a_dict[key] = value[-take_n:]

#### 3.a.1. List the 10 words whose presence most strongly predicts that the mail is ham

In [6]:
query = "ham presence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
cc	0.920928
hou	0.921190
shirley	0.921799
ect	0.922348
vince	0.922348
enron	0.922417
kaminski	0.922542
crenshaw	0.923175
713	0.923189
stinson	0.923226


#### 3.a.2. List the 10 words whose absence most strongly predicts that the mail is ham

In [7]:
query = "ham absence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
501	0.761173
unstable	0.761173
discharge	0.761173
interference	0.761174
renovation	0.761174
persistence	0.761174
cloak	0.761174
suffering	0.761174
secrecy	0.761176
php	0.761206


#### 3.a.3. List the 10 words whose presence most strongly predicts that the mail is spam

In [8]:
query = "spam presence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
click	0.750847
search	0.783951
life	0.827946
95	0.843815
save	0.860793
money	0.882610
logo	0.945333
adobe	1.003532
2005	1.020820
viagra	1.037638


#### 3.a.4. List the 10 words whose absence most strongly predicts that the mail is spam

In [9]:
query = "spam absence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
respected	0.238828
trash	0.238829
paige	0.238829
determining	0.238829
recovery	0.238831
sets	0.238833
sheet	0.238835
participating	0.238837
sometime	0.238841
steven	0.238868


We see that usually presence of certain words nearly gurantees that the document belongs to certain class. Such as `viagra`, where it's presence strongly suggests that the mail is spam. Similarly, absence of some words suggests to some level that it belongs to certain class. `secrecy` can be a great example for this, where it's absence strongly suggests that the mail is ham.<br>
<br>
Below we reimplemented our $\text{N$\dot{a}$ive Bayes}$ model to use tfidf values to discard not important words. Our implementation discards not important words based on their normalized tfidf values. Where we calculated the normalized tfidf values as follows:
1. Sum by axis 0 to get a matrix of shape (C,), where C is the distinct word count
2. Divide each column sum by document frequency (df), where df is defined as follows:
    $$ df(D, w) = |\{d \in D, w \in d\}| \text{, number of documents where word w appears} $$

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from tqdm.auto import tqdm
import pandas as pd

MODES = {
    "unigram" : (1, 1),
    "bigram" : (2, 2),
    "uni-bigram" : (1, 2)
}


class NaiveBayesV2:
    def __init__(self, mode: str = "unigram", stop_words = None, use_tfidf : bool = False, tfidf_th : float = None) -> None:
        """
        Initialize NaiveBayes model

        Args:
            mode (str, optional): Mode for BagOfWords, should be either Unigram or Bigram. Defaults to "unigram".
            stop_words (list, optional): Stop words to eliminate from BagOfWords
            use_idf (bool, optional)
            tfidf_th (float, optional): Threshold for tfidf values, necessary only if use_tfidf is True
        """
        self.count_vector = None
        self.probability_dict = dict()

        assert mode in MODES.keys(), "Mode should be either bigram or unigram"
        self.ngram_mode = MODES[mode]
        self.stop_words = stop_words
        self.use_tfidf = use_tfidf
        if use_tfidf:
            self.tfidf_th = tfidf_th

    def fit(self, x: np.ndarray, y: np.ndarray) -> None:
        """
        Fit the data, calculate probabilities according to it

        Args:
            x (np.ndarray): data with shape (N,1), each row consists of a mail
            y (np.ndarray): data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        """

        # columns is list of every words without their counts, counts stores in count_vector
        self.count_vector, columns = self.__vectorizer(x)

        self.__calculate_class_prior(y)     # calculates P(spam) and P (ham), and adds them into probability_dict
        # calculates P(x(i)|spam) and P(x(i)|ham) values, and adds them into probability_dict
        self.__calculate_likelihoods(y, columns, 1)
        
        if self.use_tfidf:
            self.__eliminate_less_important(y, columns)

    def predict(self, x_predict: np.ndarray) -> np.ndarray:
        """
        Predict the labels of the mails in x_predict

        Args:
            x_predict (np.ndarray): Data to be predicted, shape (N,1)

        Returns:
            y_predict (np.ndarray): Predictions for the mails, shape (N,)
        """

        n = x_predict.shape[0]
        y_predict = np.zeros(n)

        vector, columns = self.__vectorizer(x_predict)

        for i in range(n):
            probability_of_spam = 0
            probability_of_ham = 0

            word_idx = np.arange(columns.shape[0])[vector[i] > 0] # only work on words that the text has
            # calculate P(vj | text)
            for j in word_idx:
                if "%s|spam" % columns[j] in self.probability_dict.keys():
                    probability_of_spam += vector[i][j] * self.probability_dict[columns[j] + "|spam"]
                    probability_of_ham += vector[i][j] * self.probability_dict[columns[j] + "|ham"]

            probability_of_spam += self.probability_dict["spam"]
            probability_of_ham += self.probability_dict["ham"]

            y_predict[i] = 1 if probability_of_spam > probability_of_ham else 0

        return y_predict

    def __vectorizer(self, arr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """
        Creates a matrix which stores the words that emails includes and their counts in each email

        :param arr: an numpy array with shape (N,1) for
        :return: a CountVectorizer matrix (N, number of different words in emails) and a vector named columns
        with shape (number of different words in emails,1) which stores all words appears in mails
        """

        # initializes CountVectorizer item with ngram_mode
        vectorizer = CountVectorizer(ngram_range=self.ngram_mode, stop_words=self.stop_words)

        # vector that holds all words in emails and their counts for each item
        vector = vectorizer.fit_transform(arr)

        # convert vector variable to array for usability
        count_vector = vector.toarray()

        # names of columns
        columns = vectorizer.get_feature_names_out()

        return count_vector, columns
    
    def __eliminate_less_important(self, y : np.ndarray, columns : np.ndarray):
        tfidf = TfidfTransformer(use_idf = True, smooth_idf = True)
        
        tfidf_ham = tfidf.fit_transform(self.count_vector[y == 0]).sum(axis = 0).A1
        counts = np.sum(self.count_vector[y == 0] > 0, axis=0)
        tfidf_ham /= (1 + counts)
        
        
        tfidf_spam = tfidf.fit_transform(self.count_vector[y == 1]).sum(axis = 0).A1
        counts = np.sum(self.count_vector[y == 1] > 0, axis=0)
        
        tfidf_spam /= (1 + counts)
        
        idx_ham = np.arange(tfidf_ham.shape[0])[tfidf_ham > self.tfidf_th]
        idx_spam = np.arange(tfidf_spam.shape[0])[tfidf_spam > self.tfidf_th]
        
        old_total = tfidf_ham.shape[0] + tfidf_spam.shape[0]
        current_total = idx_ham.shape[0] + idx_spam.shape[0]
        print("%d words discarded because they fall below the tfidf threshold, predicting with %d word"\
              % ((old_total - current_total), current_total))
        
        for doc_class, idx in [("ham", idx_ham), ("spam", idx_spam)]:
            for i in idx:
                self.probability_dict["%s|%s" % (columns[i], doc_class)] = 0
        
        

    def __calculate_class_prior(self, y: np.ndarray) -> None:
        """
        Calculates class probabilities [P(spam) and P(ham)] for training examples, and adds the result into
        self.probability dictionary as "spam" and "ham" labels

        :param y: data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        :return: None
        """

        # labels are only 0 and 1 therefore if we sum all items we get number of 1s
        # instead of a for loop we can use this method
        number_of_spam = np.sum(y)
        number_of_ham = len(y) - number_of_spam

        self.probability_dict["spam"] = np.log(number_of_spam / y.shape[0])  # P(spam) = number of spams / N
        self.probability_dict["ham"] = np.log(number_of_ham / y.shape[0])     # P(ham) = number of hams / N

    def __calculate_likelihoods(self, y: np.ndarray, columns: np.ndarray, alpha: int) -> None:
        """
        Calculates likelihoods of each word that contains in emails as P(word|spam) and P(word|ham), and adds the results
        into self.probability dictionary as "word|spam" and "word|ham" labels

        :param y: data with shape (N,), each row consists of the label of the mail (0 for ham, 1 for spam)
        :param columns: a vector with shape (number of different words in emails,1) which stores all words appears in mails
        :param alpha: int value for smoothing
        :return: None
        """

        N, D = self.count_vector.shape
        spam_vector, ham_vector = np.sum(self.count_vector[y == 1], axis=0), np.sum(self.count_vector[y == 0], axis=0)

        n_s = np.sum(spam_vector) # | Text_spam |
        n_h = np.sum(ham_vector) # | Text_ham  |

        for word_i in range(D):
            n_w_s = spam_vector[word_i]
            n_h_s = ham_vector[word_i]

            self.probability_dict["%s|spam" % columns[word_i]] = np.log((n_w_s + alpha) / (n_s + D))
            self.probability_dict["%s|ham" % columns[word_i]] = np.log((n_h_s + alpha) / (n_h + D))

In [11]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

file_path = os.path.join(os.getcwd(), "emails.csv")
df = pd.read_csv(file_path)
X, y = df["text"].to_numpy(), df["spam"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

model = NaiveBayesV2(use_tfidf=True, tfidf_th = 0.3)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print("Accuracy with tfidf_th = 0.3:", accuracy_score(y_test.astype(bool), y_predict.astype(bool)))

67489 words discarded because they fall below the tfidf threshold, predicting with 107 word
Accuracy with tfidf_th = 0.3: 0.9912739965095986


We tried our model with tfidf threshold value 0.3 and noted important things:
1. Total number of used words **decreased by 67489 words**
2. Even though only 107 words were used, our accuracy is **0.9912**, which we found quite high.

We conclude that using tfidf values helps to find words that are important for certain class, hence discarding unnecessary operations.

### 3.b. Stopwords

In [12]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

word_p_a_dict_stop = find_presence_absence_probilities(X, y, ENGLISH_STOP_WORDS)

take_n = 10 # take n words with highest posterior probility
for key, value in word_p_a_dict.items():
    word_p_a_dict_stop[key].sort(key= lambda x : x[1])
    word_p_a_dict_stop[key] = value[-take_n:]

##### Highest TFIDF values for class 1 #####
['real' 'info' 'buy' 'identity' 'rate' 'remove' 'security' 'contact'
 'engines' 'sites' 'internet' 'available' 'look' 'design' 'removed' '2004'
 'today' 'creative' 'love' 'mailing' 'oem' 'getting' '19' 'hot' 'wish'
 'success' 'received' 'send' 'ready' 'day' 'credit' 'great' 'price' 'help'
 'fast' 'work' 'right' 'don' 'good' 'regards' 'address' 'thing' 'prices'
 'use' 'know' 'submit' 'hello' 'future' 'stationery' 'web' 'man' 'visit'
 'order' '2005' 'interested' 'way' 'net' 'people' 'marketing' 'home'
 'offer' 'receive' '10' 'start' 'offers' '000' 'time' 'search' 'account'
 'list' 'viagra' 'want' 'life' 'message' 'new' 'www' 'site' 'like' '95'
 'mail' 'logo' 'make' 'information' 'best' 'online' 'need' 'save' 'free'
 'http' 'just' 'adobe' 'money' 'company' 'email' 'click' 'website'
 'software' 'com' 'business' 'subject']
##### Highest TFIDF values for class 0 #####
['doc' 'office' 'data' 'market' 'options' 'london' 'make' 'send' 'just'
 'sent' 

Calculating probilities for P(class | word in doc) and P(class | word not in doc):   0%|          | 0/2 [00:00…

Doing presence spam:   0%|          | 0/100 [00:00<?, ?it/s]

Doing absence spam:   0%|          | 0/100 [00:00<?, ?it/s]

Doing presence ham:   0%|          | 0/100 [00:00<?, ?it/s]

Doing absence ham:   0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
query = "spam presence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict_stop[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
click	0.750847
search	0.783951
life	0.827946
95	0.843815
save	0.860793
money	0.882610
logo	0.945333
adobe	1.003532
2005	1.020820
viagra	1.037638


In [14]:
query = "ham presence"
print("Word\tProbability\n---------------------")
for word, prob in word_p_a_dict_stop[query]:
    print("%s\t%f" % (word, prob))

Word	Probability
---------------------
cc	0.920928
hou	0.921190
shirley	0.921799
ect	0.922348
vince	0.922348
enron	0.922417
kaminski	0.922542
crenshaw	0.923175
713	0.923189
stinson	0.923226


### 3.c. Analyzing effect of the stopwords

Even though we removed stopwords from the corpus we got the same 10 words that suggests a certain class in case of presence. We think that, this happened because of how our pipeline works. Earlier, at the elimination step by using tfidf values, we saw some stopwords in the matrix, but there were none in the output. This happened because when we are calculating their posterior probabilities, we are diving by $P(w)$ which gets higher as word seen frequently in the documents, thus resulting in low $P(w)$. At the end, even though we removed stop words, we got the same words for our results.

1. Why might it make sense to remove stop words when interpreting the model?
    1. Removing stop words helps to lower the corpus by a certain amount, thus making the model faster. Furthermore since stop words are used too often, it results in high conditional probility even though they are near to meaningless when left alone.
2. Why might it make sense to keep stop words?
    1. In our opinion, keeping stop words in lower ngram settings doesn't introduce a meaningful increase in prediction. On the other hand, if we were to keep it in higher ngram settings we think it will introduce meaningful combinations may referring to certain idioms, hence it may introduce certain word combinations that has very high conditional probility and it's presence suggesting that the mail belongs to certain class.

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import pandas as pd
import numpy as np
import os

file_path = os.path.join(os.getcwd(), "emails.csv")
df = pd.read_csv(file_path)
X, y = df["text"].to_numpy(), df["spam"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=29, stratify=y
)

args = [("unigram", None), ("unigram", ENGLISH_STOP_WORDS)]
for arg in args:
    model = NaiveBayes(*arg)        # initializes the model
    model.fit(X_train, y_train)      # training
    y_predict = model.predict(X_test)
    acc = accuracy_score(y_test.astype(bool), y_predict.astype(bool))
    print("Accuracy with ngram=%s, stop_words=%s:\t%f" % (arg[0], arg[1] != None, acc))

Accuracy with ngram=unigram, stop_words=False:	0.989529
Accuracy with ngram=unigram, stop_words=True:	0.990401


As expected, we see an increase in accuracy when we discard stop words from our model. Our accuracy increased from 0.989529 to 0.990401. We think that, the extra noise coming from calculating conditional probilities of stop words caused this difference.